In [13]:
import pandas as pd
import xml.etree.ElementTree as ET

In [14]:
tree = ET.parse("dataset/training_set.xml")
root = tree.getroot()

In [15]:
def xmlToDict(root):
    def processChild(child):
        text = child[0].text
        if len(child) > 1:
            aspectsTree = child[1]
            aspects = {aspect.attrib['category'] : aspect.attrib['polarity']
                        for aspect in aspectsTree}
            return dict({'text' : text}, **aspects)
        else:
            return {'text' : text}
    return {child.attrib['rid'] : processChild(child) for child in root}

In [17]:
def dictToAspectDf(data):
    def generateLabel(listAspects):
        aspects = ['FOOD', 'AMBIENCE', 'PRICE', 'SERVICE']
        return {aspect : (aspect in listAspects) for aspect in aspects}
    newData = {}
    for k, v in data.items():
        newData[k] = {'text' : v['text'], **generateLabel(v.keys())}
    return pd.DataFrame.from_dict(newData, orient='index')
        
def dictToSentimentDf(data):
    aspects = ['FOOD', 'AMBIENCE', 'PRICE', 'SERVICE']
    def generateLabel(dictAspects):
        listAspects = dictAspects.keys()
        return {aspect : (dictAspects[aspect] if aspect in listAspects else 'UNKNOWN') for aspect in aspects}
    newData = {}
    for k, v in data.items():
        if (len(v.keys()) == 1): continue
        newData[k] = {'text' : v['text'], **generateLabel(v)}
    return pd.DataFrame.from_dict(newData, orient='index')

In [17]:
data = xmlToDict(root)

In [18]:
dictToSentimentDf(data).to_csv('dataset/preProcessSentiment.csv')
dictToAspectDf(data).to_csv('dataset/preProcessAspect.csv')

In [18]:
tree = ET.parse("dataset/gold_submission.xml")
root = tree.getroot()

In [19]:
data = xmlToDict(root)

In [21]:
dictToSentimentDf(data).to_csv('dataset/gold_sentiment.csv')

In [10]:
df.to_csv('dataset/preprocessTest.csv')